Objetivos
Este laboratorio contiene las siguientes tareas:
•	TAREA 1: Marcar todos los sitios de lanzamiento en un mapa
•	TAREA 2: Marcar los lanzamientos exitosos/fallidos de cada sitio en el mapa
•	TAREA 3: Calcular las distancias entre un sitio de lanzamiento y sus proximidades

Después de completar las tareas anteriores, debería poder encontrar algunos patrones geográficos en los sitios de lanzamiento.
Primero, importemos los paquetes de Python necesarios para este laboratorio:


In [98]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [99]:
import folium
import pandas as pd
import numpy as np  # useful for many scientific computing in Python

In [100]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

Si necesita refrescar su memoria sobre el folium, puede descargar y consultar este laboratorio de folium anterior:

# Tarea 1: Marcar todos los sitios de lanzamiento en un mapa


Primero, intentemos agregar la ubicación de cada sitio en un mapa usando sus coordenadas de latitud y longitud.

El siguiente conjunto de datos, llamado spacex_launch_geo.csv, es un conjunto de datos ampliado con la latitud y longitud añadidas para cada sitio.

In [101]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Ahora, puedes echar un vistazo a cuáles son las coordenadas de cada sitio.

In [102]:
# Seleccione subcolumnas relevantes: `Sitio de lanzamiento`, `Lat(Latitud)`, `Long(Longitud)`, `clase`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Las coordenadas anteriores son simplemente números que no ofrecen una idea intuitiva de la ubicación de los sitios de lanzamiento. Si eres experto en geografía, puedes interpretar esos números directamente en tu mente. Si no, también funciona. Visualicemos esas ubicaciones marcándolas en un mapa.

Primero, necesitamos crear un objeto de mapa Folium, con una ubicación central inicial en el Centro Espacial Johnson de la NASA en Houston, Texas.

In [103]:
# La ubicación de inicio es el Centro Espacial Johnson de la NASA.
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

Podríamos usar folium.Circle para agregar un área circular resaltada con una etiqueta de texto en una coordenada específica. Por ejemplo,

In [104]:
# Crea un círculo azul en las coordenadas del Centro Espacial Johnson de la NASA con una etiqueta emergente que muestra su nombre
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Deberías encontrar un pequeño círculo amarillo cerca de la ciudad de Houston. Puedes ampliarlo para verlo más grande.
Ahora, agreguemos un círculo para cada sitio de lanzamiento en el marco de datos launch_sites.
TODO: Crear y agregar folium.Circle y folium.Marker para cada sitio de lanzamiento en el mapa del sitio.
jemplo de folium.Circle:
folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))
Ejemplo de folium.Marker:
folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))

In [105]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# Para cada punto de lanzamiento, agregue un objeto Círculo según sus coordenadas (Latitud, Longitud). Además, agregue el nombre del punto de lanzamiento como etiqueta emergente.

# Coordenadas centrales para inicializar el mapa (por ejemplo, centro aproximado de EE. UU.)
#nasa_coordinate = [30, -100]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Iterar sobre cada sitio de lanzamiento y agregar un círculo con etiqueta
for index, row in launch_sites_df.iterrows():
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=500,  # radio en metros
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=row['Launch Site']
    ).add_to(site_map)

# Mostrar el mapa (en Jupyter Notebook, por ejemplo)
site_map

Ahora, puedes explorar el mapa ampliando o reduciendo las áreas marcadas e intentar responder las siguientes preguntas:

¿Están todos los sitios de lanzamiento cerca de la línea del Ecuador?
NO, no todos los sitios de lanzamiento estan cerca de la linea del ecuador
¿Están todos los sitios de lanzamiento muy cerca de la costa?
SI, todos los sitios estan cerca de la costa
Intenta también explicar tus hallazgos.

In [106]:
# Tarea 2: Marque los lanzamientos exitosos/fallidos de cada sitio en el mapa

A continuación, intentemos mejorar el mapa añadiendo los resultados de lanzamiento de cada sitio y veamos cuáles tienen una alta tasa de éxito. Recordemos que el marco de datos spacex_df contiene registros detallados de lanzamiento, y la columna de clase indica si este lanzamiento fue exitoso o no.

In [107]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


A continuación, crearemos marcadores para todos los registros de lanzamiento. Si un lanzamiento fue exitoso (class = 1), usaremos un marcador verde; si fue fallido, usaremos un marcador rojo (class = 0).

Tenga en cuenta que un lanzamiento solo ocurre en uno de los cuatro sitios de lanzamiento, lo que significa que muchos registros de lanzamiento tendrán exactamente la misma coordenada. Los clústeres de marcadores pueden ser una buena manera de simplificar un mapa que contiene muchos marcadores con la misma coordenada.

Primero, creemos un objeto MarkerCluster.

In [108]:
marker_cluster = MarkerCluster()

TODO: Crear una nueva columna en el marco de datos spacex_df llamada marcador_color para almacenar los colores del marcador según el valor de class

In [109]:
# Aplicar una función para verificar el valor de la columna `class`
# Si class=1, el valor de marcador_color será verde
# Si class=0, el valor de marcador_color será rojo
# Paso 1: Crear columna marcador_color según el valor de class
spacex_df['marcador_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Paso 2: Crear mapa base centrado en el promedio de lat/long
mapa_spacex = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=5)

# Paso 3: Crear objeto MarkerCluster
marker_cluster = MarkerCluster().add_to(mapa_spacex)

# Paso 4: Agregar marcadores al cluster
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"{row['Launch Site']} - {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=row['marcador_color'])
    ).add_to(marker_cluster)

# Mostrar el mapa en Jupyter
mapa_spacex

Su mapa actualizado puede parecerse a las siguientes capturas de pantalla:

TODO: Para cada resultado de lanzamiento en el dataframe spacex_df, agregue un folium.Marker a marker_cluster

In [110]:
# Agregar marcador_cluster al mapa del sitio actual
site_map.add_child(marker_cluster)

# para cada fila en spacex_df data frame
# Crea un objeto Marker con sus coordenadas
# y personalizar la propiedad del icono Marker para indicar si este lanzamiento fue exitoso o falló,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
#TODO: Create and add a Marker cluster to the site map
#TODO: Crear y agregar un Marker cluster al mapa del sitio
# marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

A partir de los marcadores etiquetados por color en los grupos de marcadores, debería poder identificar fácilmente qué sitios de lanzamiento tienen tasas de éxito relativamente altas.

In [111]:
# TAREA 3: Calcular las distancias entre un sitio de lanzamiento y sus proximidades


A continuación, necesitamos explorar y analizar la proximidad de los puntos de lanzamiento.

Primero, agreguemos una posición del ratón en el mapa para obtener las coordenadas al pasar el ratón sobre un punto. De esta forma, mientras explora el mapa, podrá encontrar fácilmente las coordenadas de cualquier punto de interés (como una vía férrea).

In [112]:
# Agregue la posición del mouse para obtener las coordenadas (latitud, longitud) al pasar el mouse sobre el mapa
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Ahora, amplía el mapa de un sitio de lanzamiento y explora su proximidad para ver si puedes encontrar fácilmente vías férreas, carreteras, costas, etc. Mueve el ratón sobre estos puntos y anota sus coordenadas (mostradas en la esquina superior izquierda) para indicar la distancia al sitio de lanzamiento.

In [113]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

TODO: Marque un punto en la costa más cercana usando MousePosition y calcule la distancia entre el punto de la costa y el sitio de lanzamiento.

In [114]:
# Encuentra las coordenadas de la línea de costa más cercana
# p. ej.: Lat: 28.56367 Lon: -80.57163
# distance_coastline = calculate_distance(lat_sitio_lanzamiento, longitud_sitio_lanzamiento, lat_línea_costa, longitud_línea_costa)
# Coordenadas
launch_site = (28.57369, -80.64717)   # Sitio de lanzamiento
coast_point = (28.56224, -80.56705)  # Punto más cercano en la costa
distance_coastline = calculate_distance(28.5642,-80.57614,28.56224,-80.56705)

# Crear el mapa centrado en el sitio de lanzamiento
site_map = folium.Map(location=launch_site, zoom_start=15)

# Añadir marcador en el punto de la costa
folium.Marker(
    location=coast_point,
    icon=folium.Icon(color='blue'),
    popup='Punto más cercano en la costa'
).add_to(site_map)

# Añadir texto con la distancia usando DivIcon
distance_marker = folium.Marker(
    location=coast_point,
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
)
distance_marker.add_to(site_map)
# Mostrar el mapa 
site_map



In [115]:
# Crea y añade un folium.Marcador en el punto de costa más cercano seleccionado en el mapa
# Muestra la distancia entre el punto de la costa y el sitio de lanzamiento usando la propiedad del ícono 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

# Añadir línea entre sitio de lanzamiento y punto de costa
folium.PolyLine(
    [launch_site, coast_point],
    color="green",
    weight=2.5,
    opacity=1
).add_to(site_map)

# Mostrar el mapa 
site_map

TODO: Dibujar una polilínea entre un sitio de lanzamiento y el punto de la costa seleccionado

In [116]:
# Cree un objeto `folium.PolyLine` utilizando las coordenadas de la línea de costa y las coordenadas del sitio de lanzamiento
# lines=folium.PolyLine(locations=coordinates, weight=1
launch_site = (28.57369, -80.64717)   # Sitio de lanzamiento
coast_point = (28.56224, -80.56705)  # Punto más cercano en la costa
# Crear la lista de coordenadas para la línea
coordenadas = [launch_site, coast_point]

# Crear el objeto PolyLine
lines = folium.PolyLine(locations=coordenadas, weight=2, color='blue')

# Añadir la línea al mapa
site_map.add_child(lines)

Su mapa actualizado con la línea de distancia debería verse como la siguiente captura de pantalla:

Crea un marcador con la distancia a la ciudad, ferrocarril, autopista, etc. más cercana.
Dibuja una línea entre el marcador y el punto de lanzamiento.

In [117]:
# Punto de lanzamiento
launch_site = (28.5642, -80.57614)

# Punto de interés (puede ser ciudad, ferrocarril, autopista, etc.)
city_point = (28.56254, -80.58667)  # <-- Titan III Road

# Calcular la distancia
distance_to_city = calculate_distance(*launch_site, *city_point)

# Crear el mapa
site_map = folium.Map(location=launch_site, zoom_start=13)

# Añadir marcador en el punto de interés
folium.Marker(
    location=city_point,
    icon=folium.Icon(color='purple', icon='info-sign'),
    popup='Punto de interés más cercano'
).add_to(site_map)

# Añadir marcador de texto con la distancia
folium.Marker(
    location=city_point,
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#8e44ad;"><b>{:.2f} KM</b></div>'.format(distance_to_city),
    )
).add_to(site_map)

# Dibujar línea desde el sitio de lanzamiento hasta el punto de interés
folium.PolyLine(
    locations=[launch_site, city_point],
    color='purple',
    weight=2,
    opacity=0.8
).add_to(site_map)

# Mostrar el mapa
site_map